In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import scipy.stats as stats

In [2]:

# standard deviation of spherical Gaussian distribution
mu = 0.0 #zero-mean
sigma_u = 1.0
sigma_v = 1.0
sigma = [0.1, 1.0] #[0.1, 0.3, 1.0, 3, 10]

# Generate U, V from the Gaussian distribution
D = 2 #dimension
N = 15 #nbr of users = 10
M = 500 #nbr of movies : M=5*alpha
alpha_vals=[1,10] #[1,5,10,50,100]
k_best = 15 # 

#print(U.shape)


In [3]:
def condition(x):
    if x<-2:
        return 1
    elif x>=-2 and x<=-0.5:
        return 2
    elif x>-0.5 and x<=0.5:
        return 3
    elif x>0.5 and x<=2:
        return 4
    else:
        return 5

In [4]:
def similarity_list_R(R):
    df_R = pd.DataFrame(R)
    #df_R = df_R.applymap(condition)
    sim_array= cosine_similarity(df_R)[0] # for user1, if user2 we take 2nd row of cos_sim matrix
    df_R['similarity_u1']= sim_array.tolist()
    df_R= df_R.sort_values(by= 'similarity_u1', ascending= False)
    user1_sim_list= df_R.index
    return user1_sim_list

def similarity_list_U(U):
    df_U = pd.DataFrame(U)
    sim_array= cosine_similarity(df_U)[0] # for user1, if user2 we take 2nd row of cos_sim matrix
    df_U['similarity_u1']= sim_array.tolist()
    df_U= df_U.sort_values(by= 'similarity_u1', ascending= False)
    user1_sim_list= df_U.index
    return user1_sim_list

In [5]:
def ranking_similarity(u1_list_U, u1_list_R, k_best):
    tau, p_value = stats.kendalltau(u1_list_U[:k_best], u1_list_R[:k_best])
    return tau


In [6]:
data= []
for s in sigma:
    for alpha in alpha_vals:
        U = np.random.normal(mu, sigma_u, size=(D,N))
        V = np.random.normal(mu, sigma_v, size=(D,M*alpha))
        noise = np.random.normal(mu, s, size=(N,M*alpha)) #add noise
        R = np.matmul(U.T,V) +noise
        similarities= ranking_similarity(similarity_list_U(U.T), similarity_list_R(R), k_best)
        row= [M*alpha, s, similarities]
        data.append(row)
        print('For sigma^2= {}, for M=: {}'.format(s,M*alpha))
        print('similarity between U and R for user1\'s similar users:{:.5f}'.format(similarities))

For sigma^2= 0.1, for M=: 500
similarity between U and R for user1's similar users:0.98095
For sigma^2= 0.1, for M=: 5000
similarity between U and R for user1's similar users:1.00000
For sigma^2= 1.0, for M=: 500
similarity between U and R for user1's similar users:0.56190
For sigma^2= 1.0, for M=: 5000
similarity between U and R for user1's similar users:-0.10476


In [7]:
df = pd.DataFrame(data, columns=['M','sigma','sim_U_R'])
df

,M,sigma,sim_U_R
0,500,0.1,0.980952
1,5000,0.1,1.000000
2,500,1.0,0.561905
3,5000,1.0,-0.104762
